In [3]:
!pip install catboost

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import time



from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb

from sklearn.model_selection import GridSearchCV

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
df_train = pd.read_csv('/kaggle/input/blackfriday-scaled/BF_train_scaled.csv',index_col='Unnamed: 0')
df_test = pd.read_csv('/kaggle/input/blackfriday-scaled/BF_test_scaled.csv',index_col='Unnamed: 0')

In [5]:
df_train_randomimp = pd.read_csv('/kaggle/input/blackfriday-randomimpute/BF_train_randomimpute.csv',index_col='Unnamed: 0')
df_test_random_imp = pd.read_csv('/kaggle/input/blackfriday-randomimpute/BF_test_randomimpute.csv',index_col='Unnamed: 0')

In [5]:
df_train

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,0,1,0.294864,1,2,0,-0.610809,-0.293605,0.368792,8370.0
1,0,1,0.294864,1,2,0,-1.118912,-0.759362,-0.361176,15200.0
2,0,1,0.294864,1,2,0,1.675656,-0.293605,0.368792,1422.0
3,0,1,0.294864,1,2,0,1.675656,1.103664,0.368792,1057.0
4,1,7,1.214734,3,4,0,0.659449,-0.293605,0.368792,7969.0
...,...,...,...,...,...,...,...,...,...,...
550063,1,6,0.754799,2,1,1,3.708069,-0.293605,0.368792,368.0
550064,0,3,-1.084943,3,3,0,3.708069,-0.293605,0.368792,371.0
550065,0,3,1.061423,2,4,1,3.708069,-0.293605,0.368792,137.0
550066,0,7,-1.084943,3,2,0,3.708069,-0.293605,0.368792,365.0


In [6]:
df_test.head()

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
550068,1,5,-0.165072,2,2,1,-1.118912,0.405029,0.368792,NaN
550069,1,3,1.368046,3,0,0,-0.610809,-0.992240,0.368792,NaN
550070,0,4,-1.084943,2,4,1,-0.102705,1.103664,0.368792,NaN
550071,0,4,-1.084943,2,4,1,-0.356757,-0.060727,0.368792,NaN
550072,0,3,-1.084943,3,1,0,-0.356757,-0.992240,-1.091145,NaN


In [7]:
X = df_train.drop('Purchase',axis=1)
y = df_train['Purchase']

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

### **Training with only basic model objects**


In [12]:
models = {
        'RandomForestRegressor': RandomForestRegressor(),
        'LinearRegression': LinearRegression(),
        'Ridge': Ridge(),
        'Lasso': Lasso(),
        'XGBRegressor': XGBRegressor(),
        'CatBoostRegressor' : CatBoostRegressor(verbose=False),
        'Gradient_Boosting' : GradientBoostingRegressor(),
        'Adaboost_Regressor' : AdaBoostRegressor(),
        
    }


In [13]:
def evaluate_default_models(X_train, y_train, X_test, y_test,models):
    
    
    report_nogscv = []

    for model_name, model in models.items():
        start_time = time.time()
        model.fit(X_train, y_train)
        end_time = time.time()
        
        
        training_time = end_time - start_time
        
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        train_model_score = r2_score(y_train, y_train_pred)
        test_model_score = r2_score(y_test, y_test_pred)
        Mae = mean_absolute_error(y_test, y_test_pred)
        RMse = mean_squared_error(y_test, y_test_pred,squared= False)

        report_nogscv.append({
            'Model_name': model_name,
            'R2_Score_test': test_model_score,
            'R2_Score_train': train_model_score,
            'MAE': Mae,
            'RMSE': RMse,
            'Training_Time': training_time/60
            
        })

    return report_nogscv

In [10]:
report_nogs = evaluate_default_models(X_train, y_train, X_test, y_test,models)

In [11]:
report_nogs

[{'Model_name': 'RandomForestRegressor',
  'R2_Score_test': 0.6293874221433616,
  'R2_Score_train': 0.7807725916367578,
  'MAE': 2221.358179714615,
  'RMSE': 3051.5698125845556,
  'Training_Time': 1.300696337223053},
 {'Model_name': 'LinearRegression',
  'R2_Score_test': 0.13009752050368628,
  'R2_Score_train': 0.13116158704004344,
  'MAE': 3576.6193901056677,
  'RMSE': 4675.181946658147,
  'Training_Time': 0.0030258615811665853},
 {'Model_name': 'Ridge',
  'R2_Score_test': 0.13009752510732786,
  'R2_Score_train': 0.13116158703917002,
  'MAE': 3576.6198588074476,
  'RMSE': 4675.1819342873005,
  'Training_Time': 0.0014978845914204915},
 {'Model_name': 'Lasso',
  'R2_Score_test': 0.1300963059940763,
  'R2_Score_train': 0.13116097207556932,
  'MAE': 3576.6801082617985,
  'RMSE': 4675.185210271918,
  'Training_Time': 0.002513424555460612},
 {'Model_name': 'XGBRegressor',
  'R2_Score_test': 0.6660315347704567,
  'R2_Score_train': 0.6795480631107598,
  'MAE': 2163.14527613874,
  'RMSE': 2896

In [12]:
report_nogs_df = pd.DataFrame(report_nogs).sort_values(by='RMSE',ascending=True)

In [13]:
report_nogs_df

,Model_name,R2_Score_test,R2_Score_train,MAE,RMSE,Training_Time
4,XGBRegressor,0.666032,0.679548,2163.145276,2896.783001,0.465376
5,CatBoostRegressor,0.665757,0.677714,2165.890656,2897.973900,0.950537
6,Gradient_Boosting,0.642872,0.649617,2267.903471,2995.540704,0.421802
0,RandomForestRegressor,0.629387,0.780773,2221.358180,3051.569813,1.300696
7,Adaboost_Regressor,0.437517,0.441683,2983.203713,3759.394090,0.214022
2,Ridge,0.130098,0.131162,3576.619859,4675.181934,0.001498
1,LinearRegression,0.130098,0.131162,3576.619390,4675.181947,0.003026
3,Lasso,0.130096,0.131161,3576.680108,4675.185210,0.002513


In [14]:
report_nogs_df.to_csv('/kaggle/working/Default_Model_Report.csv')

### **XGB, catboost  performed better, we will try using hyperparameter tuning to see if we can improve the score**

## **Training with hyperparameter tuning**

In [13]:
models_params= {


                "XGBRegressor":
                            {
                            'model_obj' : XGBRegressor(tree_method = 'gpu_hist'),
                            'params' :              {
                                                        'learning_rate':[0.1,0.15,0.05,0.3],
                                                        'max_depth': [10,15,13],
                                                        'gamma': [0,0.1,0.2],
                                                        'n_estimators' : [100,125,150,200]
                                                        
                                
                                                    },
                            },


                  "CatBoosting Regressor": 
                            {
                            'model_obj' : CatBoostRegressor(verbose=False,task_type='GPU'),
                            'params' :          {
                                                    'depth': [5,10,15],
                                                    'learning_rate': [0.1,0.05,0.2],
                                                    'iterations': [1000,1500]
                                                    },
                            },



            }

In [7]:
class ModelEvaluator:
    def __init__(self, X_train, y_train, X_test, y_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.best_models = {}

    def evaluate_models_hypeparam(self, models_params):
        report = []

        for model_name, m_p in models_params.items():
            model = m_p['model_obj']
            params = m_p['params']

            gs = GridSearchCV(model, params, cv=4, scoring='neg_mean_squared_error')

            start_time = time.time()
            gs.fit(self.X_train, self.y_train)
            end_time = time.time()

            training_time = end_time - start_time

            best_model = gs.best_estimator_

            y_train_pred = best_model.predict(self.X_train)
            y_test_pred = best_model.predict(self.X_test)

            test_model_score = r2_score(self.y_test, y_test_pred)
            train_model_score = r2_score(self.y_train, y_train_pred)

            RMSE = mean_squared_error(self.y_test, y_test_pred, squared=False)

            report.append({
                'Model_name': model_name,
                'RMSE': RMSE,
                'RMSE_GS': np.sqrt(-gs.best_score_),
                'R2_test': test_model_score,
                'R2_train': train_model_score,
                'Best_Params': gs.best_params_,
                'Training_Time': training_time/60
            })

            self.best_models[model_name] = best_model

        return report

    def get_best_model(self, model_name):
        return self.best_models.get(model_name)

In [18]:
evaluator = ModelEvaluator(X_train,y_train,X_test,y_test)
report_gs = evaluator.evaluate_models_hypeparam(models_params)

In [26]:
report_hyperparam_df = pd.DataFrame(report_gs)

In [32]:
report_hyperparam_df

,Model_name,RMSE,RMSE_GS,R2_test,R2_train,Best_Params,Training_Time
0,XGBRegressor,2873.666958,2865.491200,0.671340,0.712586,"{'gamma': 0, 'learning_rate': 0.15, 'max_depth...",80.993101
1,CatBoosting Regressor,2877.370389,2865.265041,0.670493,0.702945,"{'depth': 10, 'iterations': 1500, 'learning_ra...",100.988060


In [27]:
report_hyperparam_df.to_csv('/kaggle/working/HyperParam_Report.csv')

### **As we can see,not a drastic improvement has occured from the default model training results. We can try further hyper parameter optimizationonly for XGB Regressor**

In [19]:
models_params_XGB = {
    "XGBRegressor": {
        'model_obj': XGBRegressor(tree_method = 'gpu_hist'),
        'params': {
            'learning_rate': [0.1,0.15,0.2,0.25,0.3],
            'max_depth': [5,7,10,12,15,18,20],
#             'subsample': [0.5,0.3,0.8,1.0],
#             'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
            'n_estimators' : [100,125,150,200,250]
        },

}
}


In [ ]:
evaluator_xgb = ModelEvaluator(X_train,y_train,X_test,y_test)
report_gs_xgb = evaluator_xgb.evaluate_models_hypeparam(models_params_XGB)

In [17]:
report_gs_xgb

[{'Model_name': 'XGBRegressor',
  'RMSE': 2877.207857944487,
  'RMSE_GS': 2863.6662384604115,
  'R2_test': 0.6705298984579418,
  'R2_train': 0.7027892052331817,
  'Best_Params': {'colsample_bytree': 0.9,
   'learning_rate': 0.1,
   'max_depth': 10,
   'subsample': 0.8},
  'Training_Time': 61.73541598717372}]

In [15]:
pd.DataFrame(report_gs_xgb)


,Model_name,RMSE,RMSE_GS,R2_test,R2_train,Best_Params,Training_Time
0,XGBRegressor,2877.207858,2863.666238,0.67053,0.702789,"{'colsample_bytree': 0.9, 'learning_rate': 0.1...",61.735416


In [16]:
evaluator_xgb.get_best_model('XGBRegressor')

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [18]:
report_gs_xgb

[{'Model_name': 'XGBRegressor',
  'RMSE': 2875.617598639268,
  'RMSE_GS': 2861.7854095176567,
  'R2_test': 0.6708940001333134,
  'R2_train': 0.711584450107835,
  'Best_Params': {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 150},
  'Training_Time': 8.35694143374761}]

In [15]:
report_gs_xgb

[{'Model_name': 'XGBRegressor',
  'RMSE': 2879.458102727801,
  'RMSE_GS': 2867.6814253114435,
  'R2_test': 0.6700143442600622,
  'R2_train': 0.7010191952048999,
  'Best_Params': {'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 200},
  'Training_Time': 1.6281537254651388}]

In [31]:
from pandas.io.json import json_normalize
data = [
    {
        'Model_name': 'XGBRegressor',
        'RMSE': 2875.617598639268,
        'RMSE_GS': 2861.7854095176567,
        'R2_test': 0.6708940001333134,
        'R2_train': 0.711584450107835,
        'Best_Params': {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 150},
        'Training_Time': 8.35694143374761
    }
]

df88 = pd.DataFrame(data)
params_df = pd.json_normalize(df88['Best_Params'])
df88 = pd.concat([df88, params_df], axis=1)
df88.drop('Best_Params', axis=1, inplace=True)
df88.to_csv('/kaggle/working/RXGB_HyperParam_Report.csv', index=False)

,Model_name,RMSE,RMSE_GS,R2_test,R2_train,Training_Time,learning_rate,max_depth,n_estimators
0,XGBRegressor,2875.617599,2861.78541,0.670894,0.711584,8.356941,0.1,10,150


### **Training dataset with Random Imputation**

In [7]:
df_train_randomimp

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,0,1,10,1,2,0,3,6,5,8370.0
1,0,1,10,1,2,0,1,6,14,15200.0
2,0,1,10,1,2,0,12,5,17,1422.0
3,0,1,10,1,2,0,12,14,14,1057.0
4,1,7,16,3,4,0,8,16,16,7969.0
...,...,...,...,...,...,...,...,...,...,...
550063,1,6,13,2,1,1,20,2,15,368.0
550064,0,3,1,3,3,0,20,4,14,371.0
550065,0,3,15,2,4,1,20,8,17,137.0
550066,0,7,1,3,2,0,20,10,15,365.0


In [8]:
df_test_random_imp

,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
550068,1,5,7,2,2,1,1,11,8,NaN
550069,1,3,17,3,0,0,3,5,6,NaN
550070,0,4,1,2,4,1,5,14,11,NaN
550071,0,4,1,2,4,1,4,9,11,NaN
550072,0,3,1,3,1,0,4,5,12,NaN
...,...,...,...,...,...,...,...,...,...,...
783662,0,3,15,2,4,1,8,15,16,NaN
783663,0,3,15,2,4,1,5,8,16,NaN
783664,0,3,15,2,4,1,1,5,12,NaN
783665,0,5,1,3,4,0,10,16,18,NaN


In [9]:
X_random = df_train_randomimp.drop('Purchase',axis=1)
y_random = df_train_randomimp['Purchase']

In [10]:
from sklearn.model_selection import train_test_split
X_train_random,X_test_random,y_train_random,y_test_random = train_test_split(X_random,y_random,test_size=0.2,random_state=42)

In [14]:
report_nogs = evaluate_default_models(X_train_random, y_train_random, X_test_random, y_test_random,models)

In [16]:
report_nogs_df = pd.DataFrame(report_nogs).sort_values(by='RMSE',ascending=True)
report_nogs_df

,Model_name,R2_Score_test,R2_Score_train,MAE,RMSE,Training_Time
5,CatBoostRegressor,0.662969,0.675674,2174.640750,2910.033970,0.623863
4,XGBRegressor,0.662287,0.676991,2175.771410,2912.978050,0.339129
6,Gradient_Boosting,0.639306,0.646631,2278.403477,3010.460335,0.716410
0,RandomForestRegressor,0.606103,0.843910,2311.210054,3145.971139,2.351232
7,Adaboost_Regressor,0.403044,0.407753,3055.540756,3872.883350,0.303605
2,Ridge,0.125885,0.126704,3589.581625,4686.489246,0.001332
1,LinearRegression,0.125885,0.126704,3589.581674,4686.489246,0.003847
3,Lasso,0.125884,0.126704,3589.516827,4686.491954,0.001958


In [30]:
data = {
    'Model_name': ['CatBoostRegressor', 'XGBRegressor', 'Gradient_Boosting', 'RandomForestRegressor',
                   'Adaboost_Regressor', 'Ridge', 'LinearRegression', 'Lasso'],
    'R2_Score_test': [0.662969, 0.662287, 0.639306, 0.606103, 0.403044, 0.125885, 0.125885, 0.125884],
    'R2_Score_train': [0.675674, 0.676991, 0.646631, 0.843910, 0.407753, 0.126704, 0.126704, 0.126704],
    'MAE': [2174.640750, 2175.771410, 2278.403477, 2311.210054, 3055.540756, 3589.581625, 3589.581674, 3589.516827],
    'RMSE': [2910.033970, 2912.978050, 3010.460335, 3145.971139, 3872.883350, 4686.489246, 4686.489246, 4686.491954],
    'Training_Time': [0.623863, 0.339129, 0.716410, 2.351232, 0.303605, 0.001332, 0.003847, 0.001958]
}

df_RIDR = pd.DataFrame(data)
df_RIDR.to_csv('/kaggle/working/RandomImpute_Default_Report.csv', index=False)

### Again, CatBoost and XGBoost performed better. Lets try using hyperparameter tuning for these two

In [14]:
evaluator = ModelEvaluator(X_train_random, y_train_random, X_test_random, y_test_random)
report_gs = evaluator.evaluate_models_hypeparam(models_params)

In [19]:
random_impute_hyper_df=pd.DataFrame(report_gs)

In [21]:
random_impute_hyper_df


,Model_name,RMSE,RMSE_GS,R2_test,R2_train,Best_Params,Training_Time
0,XGBRegressor,2890.396364,2878.656628,0.667503,0.710068,"{'gamma': 0, 'learning_rate': 0.1, 'max_depth'...",81.967924


In [20]:
random_impute_hyper_df.to_csv('/kaggle/working/RandomImpute_HyperParam_Report.csv')